In [1]:
%%capture
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

In [2]:
%%capture
!pip install stable-baselines3[extra]
!pip install gymnasium

In [3]:
%%capture
!pip install huggingface_sb3
!pip install huggingface_hub
!pip install panda_gym

In [1]:
import os

import gymnasium as gym
import panda_gym

from huggingface_sb3 import load_from_hub, package_to_hub

from stable_baselines3 import A2C, PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor, FlattenExtractor
from huggingface_hub import notebook_login

In [2]:
env_id = "PandaPickAndPlace-v3"
env = make_vec_env(env_id, n_envs=10)

pybullet build time: Nov 28 2023 23:45:17


argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color

In [28]:
s_size = env.observation_space.shape
a_size = env.action_space

In [17]:
print("_____OBSERVATION SPACE_____ \n")
print("The State Space is: ", s_size)
print("Sample observation", env.observation_space.sample())

_____OBSERVATION SPACE_____ 

The State Space is:  None
Sample observation OrderedDict([('achieved_goal', array([-4.530908 ,  7.8015103, -0.8189112], dtype=float32)), ('desired_goal', array([-2.127316 ,  6.7671833, -8.035298 ], dtype=float32)), ('observation', array([ 1.9598203 , -3.657007  ,  4.6873617 , -8.044689  , -9.165232  ,
        2.7746022 ,  1.5941757 ,  7.0314083 , -8.070028  , -8.71118   ,
       -0.12170128, -2.4668517 ,  7.705765  ,  0.08872256, -9.827541  ,
       -4.7936187 ,  8.407485  , -7.717925  ,  0.65760267], dtype=float32))])


In [18]:
print("\n _____ACTION SPACE_____ \n")
print("The Action Space is: ", a_size)
print("Action Space Sample", env.action_space.sample())


 _____ACTION SPACE_____ 

The Action Space is:  Box(-1.0, 1.0, (4,), float32)
Action Space Sample [-0.5572468   0.12877037  0.18574806  0.75657105]


In [19]:
env = VecNormalize(env,norm_obs=True, norm_reward=True, clip_obs=10.)

---
---

In [20]:
policy_kwargs = dict(
    net_arch=[dict(pi=[64, 64], vf=[64, 64])]
)

In [21]:
model = A2C(policy = "MultiInputPolicy",
            env = env
            ,policy_kwargs=policy_kwargs, verbose=1,
            learning_rate=0.0007, n_steps=5, gamma=0.99, gae_lambda=0.95,
            ent_coef=0.0, vf_coef=0.5, max_grad_norm=0.5)

Using cuda device


/home/tomek/pytorch_learning/venv/lib/python3.10/site-packages/stable_baselines3/common/policies.py:486: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


In [22]:
model.learn(1_000_000)

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 49       |
|    ep_rew_mean        | -48.9    |
|    success_rate       | 0.03     |
| time/                 |          |
|    fps                | 340      |
|    iterations         | 100      |
|    time_elapsed       | 14       |
|    total_timesteps    | 5000     |
| train/                |          |
|    entropy_loss       | -5.74    |
|    explained_variance | 0.982    |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | -1.39    |
|    std                | 1.01     |
|    value_loss         | 0.107    |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 49.6     |
|    ep_rew_mean        | -49.6    |
|    success_rate       | 0.01     |
| time/                 |          |
|    fps                | 345      |
|    iterations         | 200      |
|

In [23]:
model.save("a2c-PandaPickAndPlace-v3")
env.save("PandaPickAndPlace_vec_normalize.pkl")

In [24]:
eval_env = DummyVecEnv([lambda: gym.make("PandaPickAndPlace-v3")])
eval_env = VecNormalize.load("PandaPickAndPlace_vec_normalize.pkl", eval_env)

# We need to override the render_mode
eval_env.render_mode = "rgb_array"

#  do not update them at test time
eval_env.training = False
# reward normalization is not needed at test time
eval_env.norm_reward = False

# Load the agent
model = A2C.load("a2c-PandaPickAndPlace-v3")

mean_reward, std_reward = evaluate_policy(model, eval_env)

print(f"Mean reward = {mean_reward:.2f} +/- {std_reward:.2f}")

argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886


/home/tomek/pytorch_learning/venv/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Mean reward = -50.00 +/- 0.00


In [7]:
notebook_login()
!git config --global credential.helper store

In [26]:
from huggingface_sb3 import package_to_hub

package_to_hub(
    model=model,
    model_name=f"a2c-{env_id}",
    model_architecture="A2C",
    env_id=env_id,
    eval_env=eval_env,
    repo_id=f"Hevagog/a2c-{env_id}",
    commit_message="Initial commit",
)

ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: if you encounter a bug, please open an issue.


/home/tomek/pytorch_learning/venv/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Saving video to /tmp/tmphpv17ng4/-step-0-to-step-1000.mp4
Moviepy - Building video /tmp/tmphpv17ng4/-step-0-to-step-1000.mp4.
Moviepy - Writing video /tmp/tmphpv17ng4/-step-0-to-step-1000.mp4



Moviepy - Done !
Moviepy - video ready /tmp/tmphpv17ng4/-step-0-to-step-1000.mp4


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

...

pytorch_variables.pth:   0%|          | 0.00/864 [00:00<?, ?B/s]

vec_normalize.pkl:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

policy.optimizer.pth:   0%|          | 0.00/52.1k [00:00<?, ?B/s]

ℹ Your model is pushed to the Hub. You can view your model here:
https://huggingface.co/Hevagog/a2c-PandaPickAndPlace-v3/tree/main/


CommitInfo(commit_url='https://huggingface.co/Hevagog/a2c-PandaPickAndPlace-v3/commit/1b60fbdee9f768b6abe8723069a63e3e74327c83', commit_message='Initial commit', commit_description='', oid='1b60fbdee9f768b6abe8723069a63e3e74327c83', pr_url=None, pr_revision=None, pr_num=None)

---
---

In [130]:
env = make_vec_env(env_id, n_envs=16)
env = VecNormalize(env,norm_obs=True, norm_reward=True, clip_obs=20.)

argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color

In [134]:
policy2 = dict(
    net_arch=[dict(pi=[512, 1024, 1024], vf=[512, 1024, 1024])]
)

In [135]:
model2 = A2C(
    policy="MultiInputPolicy",
    env=env,
    policy_kwargs=policy2,
    verbose=1,
    learning_rate=0.0015,
    n_steps=25,
    gamma=0.99,
    gae_lambda=0.95,
    ent_coef=0.01,
    vf_coef=0.5,
    max_grad_norm=0.5,
    tensorboard_log="./a2c_tensorboard/",
)

Using cuda device


In [136]:
model2.learn(1_500_000)

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 47.5     |
|    ep_rew_mean        | -47.5    |
|    success_rate       | 0.05     |
| time/                 |          |
|    fps                | 517      |
|    iterations         | 100      |
|    time_elapsed       | 77       |
|    total_timesteps    | 40000    |
| train/                |          |
|    entropy_loss       | -5.82    |
|    explained_variance | 0.361    |
|    learning_rate      | 0.0015   |
|    n_updates          | 99       |
|    policy_loss        | 0.0124   |
|    std                | 1.04     |
|    value_loss         | 0.146    |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 48       |
|    ep_rew_mean        | -48      |
|    success_rate       | 0.04     |
| time/                 |          |
|    fps                | 507      |
|    iterations         | 200      |
|

KeyboardInterrupt: 

In [120]:
model2.save("a2c-PandaPickAndPlace_v3_2")
env.save("PandaPickAndPlace_vec_normalize_2.pkl")

In [121]:
eval_env = DummyVecEnv([lambda: gym.make("PandaPickAndPlace-v3")])
eval_env = VecNormalize.load("PandaPickAndPlace_vec_normalize_2.pkl", eval_env)

# We need to override the render_mode
eval_env.render_mode = "rgb_array"

#  do not update them at test time
eval_env.training = False
# reward normalization is not needed at test time
eval_env.norm_reward = False

# Load the agent
model = A2C.load("a2c-PandaPickAndPlace_v3_2")

mean_reward, std_reward = evaluate_policy(model, eval_env)

print(f"Mean reward = {mean_reward:.2f} +/- {std_reward:.2f}")

argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886


/home/tomek/pytorch_learning/venv/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Mean reward = -50.00 +/- 0.00


In [122]:
from huggingface_sb3 import package_to_hub

package_to_hub(
    model=model2,
    model_name=f"a2c-{env_id}",
    model_architecture="A2C",
    env_id=env_id,
    eval_env=eval_env,
    repo_id=f"Hevagog/a2c-{env_id}",
    commit_message="Switched policy network architecture to 512x512x512",
)

ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: if you encounter a bug, please open an issue.
Moviepy - Building video /tmp/tmp3eymr0sb/-step-0-to-step-1000.mp4.
Moviepy - Writing video /tmp/tmp3eymr0sb/-step-0-to-step-1000.mp4



t:   0%|          | 0/31 [00:00<?, ?it/s, now=None]Exception ignored in: <function VecVideoRecorder.__del__ at 0x7f6794b19e10>
Traceback (most recent call last):
  File "/home/tomek/pytorch_learning/venv/lib/python3.10/site-packages/stable_baselines3/common/vec_env/vec_video_recorder.py", line 113, in __del__
    self.close_video_recorder()
  File "/home/tomek/pytorch_learning/venv/lib/python3.10/site-packages/stable_baselines3/common/vec_env/vec_video_recorder.py", line 104, in close_video_recorder
    self.video_recorder.close()
  File "/home/tomek/pytorch_learning/venv/lib/python3.10/site-packages/gymnasium/wrappers/monitoring/video_recorder.py", line 157, in close
    clip.write_videofile(self.path, logger=moviepy_logger)
  File "<decorator-gen-69>", line 2, in write_videofile
  File "/home/tomek/pytorch_learning/venv/lib/python3.10/site-packages/moviepy/decorators.py", line 54, in requires_duration
    return f(clip, *a, **k)
  File "<decorator-gen-68>", line 2, in write_videofile

ℹ Pushing repo Hevagog/a2c-PandaPickAndPlace-v3 to the Hugging Face
Hub


/home/tomek/pytorch_learning/venv/lib/python3.10/site-packages/gymnasium/wrappers/monitoring/video_recorder.py:178: UserWarning: WARN: Unable to save last video! Did you call close()?
  logger.warn("Unable to save last video! Did you call close()?")


policy.optimizer.pth:   0%|          | 0.00/4.32M [00:00<?, ?B/s]

...

policy.pth:   0%|          | 0.00/4.33M [00:00<?, ?B/s]

a2c-PandaPickAndPlace-v3.zip:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

vec_normalize.pkl:   0%|          | 0.00/3.09k [00:00<?, ?B/s]

ℹ Your model is pushed to the Hub. You can view your model here:
https://huggingface.co/Hevagog/a2c-PandaPickAndPlace-v3/tree/main/


CommitInfo(commit_url='https://huggingface.co/Hevagog/a2c-PandaPickAndPlace-v3/commit/0e7d10460b6dbe6ab9b45d7ba058260cd15a7030', commit_message='Switched policy network architecture to 512x512x512', commit_description='', oid='0e7d10460b6dbe6ab9b45d7ba058260cd15a7030', pr_url=None, pr_revision=None, pr_num=None)

---
---

# PPO

In [13]:
env_id = "PandaPickAndPlaceDense-v3"

In [14]:
env = make_vec_env(env_id, n_envs=16)
env = VecNormalize(env,norm_obs=True, norm_reward=True, clip_obs=20.)

argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color

In [17]:
import torch as th

In [21]:
policy_ppo = {
    "net_arch":  [dict(pi=[1024, 1024,1024], vf=[1024, 1024,1024])],
    "activation_fn": th.nn.ReLU
}

In [24]:
model_ppo = PPO(
    policy="MultiInputPolicy",
    env=env,
    policy_kwargs=policy_ppo,
    verbose=1,
    learning_rate=1.5e-3,  # Adjusted learning rate
    n_steps=2048,  # Increased n_steps for more stable updates
    batch_size=2048,  # Appropriate batch size
    n_epochs=20,  
    gamma=0.99,
    gae_lambda=0.95,
    clip_range=0.2,  
    ent_coef=0.01,
    vf_coef=0.5,
    max_grad_norm=0.5,
    tensorboard_log="./ppo" + env_id + "/",
)

Using cuda device


In [25]:
model_ppo.learn(4_000_000)

Logging to ./ppoPandaPickAndPlaceDense-v3/PPO_2
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 47.5     |
|    ep_rew_mean     | -10.3    |
|    success_rate    | 0.07     |
| time/              |          |
|    fps             | 498      |
|    iterations      | 1        |
|    time_elapsed    | 65       |
|    total_timesteps | 32768    |
---------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 47.1      |
|    ep_rew_mean          | -10.4     |
|    success_rate         | 0.06      |
| time/                   |           |
|    fps                  | 489       |
|    iterations           | 2         |
|    time_elapsed         | 133       |
|    total_timesteps      | 65536     |
| train/                  |           |
|    approx_kl            | 7.9183598 |
|    clip_fraction        | 0.18      |
|    clip_range           | 0.2       |
|    entropy_loss 

KeyboardInterrupt: 

---
---
TQC
---

In [33]:
from sb3_contrib import TQC

In [34]:
env_id = "PandaPickAndPlace-v3"

In [35]:
env = make_vec_env(env_id, n_envs=16)
env = VecNormalize(env,norm_obs=True, norm_reward=True, clip_obs=10.)

argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color

In [36]:
policy_tqc = dict(net_arch=[2048, 2048, 2048], n_critics=4, n_quantiles=50)

In [37]:
model = TQC(
    "MultiInputPolicy",
    env,
    policy_kwargs=policy_tqc,
    verbose=1,
    learning_rate=1e-5,
    ent_coef='auto',
    buffer_size=1_000_000,
    learning_starts=5_000,
    batch_size=2048,
    tau=0.008,
    gamma=0.99,
    train_freq=1,
    gradient_steps=1,
    target_entropy="auto",
    action_noise=None,
    tensorboard_log="./tqc" + env_id + "/",
)

Using cuda device


In [38]:
model.learn(5_000_000)

Logging to ./tqcPandaPickAndPlace-v3/TQC_11
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 46.9     |
|    ep_rew_mean     | -46.9    |
|    success_rate    | 0.0625   |
| time/              |          |
|    episodes        | 4        |
|    fps             | 970      |
|    time_elapsed    | 0        |
|    total_timesteps | 800      |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 46.9     |
|    ep_rew_mean     | -46.9    |
|    success_rate    | 0.0625   |
| time/              |          |
|    episodes        | 8        |
|    fps             | 968      |
|    time_elapsed    | 0        |
|    total_timesteps | 800      |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 46.9     |
|    ep_rew_mean     | -46.9    |
|    success_rate    | 0.0625   |
| time/              |          |
|   

KeyboardInterrupt: 

In [19]:
model.save("tqc-PandaPickAndPlace-v6")
env.save("PandaPickAndPlace_vec_normalize_tqc6.pkl")

In [20]:
eval_env = DummyVecEnv([lambda: gym.make("PandaPickAndPlace-v3")])
eval_env = VecNormalize.load("PandaPickAndPlace_vec_normalize_tqc6.pkl", eval_env)

# We need to override the render_mode
eval_env.render_mode = "rgb_array"

#  do not update them at test time
eval_env.training = False
# reward normalization is not needed at test time
eval_env.norm_reward = False

# Load the agent
model = TQC.load("tqc-PandaPickAndPlace-v6")

mean_reward, std_reward = evaluate_policy(model, eval_env)

print(f"Mean reward = {mean_reward:.2f} +/- {std_reward:.2f}")

argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886


/home/tomek/pytorch_learning/venv/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Mean reward = -20.20 +/- 20.10


In [21]:
from huggingface_sb3 import package_to_hub

package_to_hub(
    model=model,
    model_name=f"tqc-{env_id}",
    model_architecture="TQC",
    env_id=env_id,
    eval_env=eval_env,
    repo_id=f"Hevagog/tqc-{env_id}",
    commit_message="TQC upgrade for PandaPickAndPlace-v3",
)

ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: if you encounter a bug, please open an issue.
Saving video to /tmp/tmpjkmmh9ra/-step-0-to-step-1000.mp4
Moviepy - Building video /tmp/tmpjkmmh9ra/-step-0-to-step-1000.mp4.
Moviepy - Writing video /tmp/tmpjkmmh9ra/-step-0-to-step-1000.mp4



ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

Moviepy - Done !
Moviepy - video ready /tmp/tmpjkmmh9ra/-step-0-to-step-1000.mp4


frame= 1001 fps=0.0 q=-1.0 Lsize=     800kB time=00:00:39.92 bitrate= 164.2kbits/s speed=56.5x    
video:788kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 1.578572%
[libx264 @ 0x563c9ad73ac0] frame I:5     Avg QP:16.30  size:  6999
[libx264 @ 0x563c9ad73ac0] frame P:272   Avg QP:24.00  size:  1264
[libx264 @ 0x563c9ad73ac0] frame B:724   Avg QP:28.80  size:   590
[libx264 @ 0x563c9ad73ac0] consecutive B-frames:  1.5%  4.6%  4.8% 89.1%
[libx264 @ 0x563c9ad73ac0] mb I  I16..4: 27.9% 55.5% 16.7%
[libx264 @ 0x563c9ad73ac0] mb P  I16..4:  0.2%  0.7%  0.9%  P16..4:  2.3%  1.7%  1.1%  0.0%  0.0%    skip:93.2%
[libx264 @ 0x563c9ad73ac0] mb B  I16..4:  0.0%  0.1%  0.1%  B16..8:  3.5%  1.7%  0.7%  direct: 0.2%  skip:93.7%  L0:49.9% L1:45.6% BI: 4.5%
[libx264 @ 0x563c9ad73ac0] 8x8 transform intra:47.1% inter:8.4%
[libx264 @ 0x563c9ad73ac0] coded y,uvDC,uvAC intra: 23.0% 4.2% 2.5% inter: 1.0% 0.1% 0.0%
[libx264 @ 0x563c9ad73ac0] i16 v,h,dc,p: 77% 11% 12%  1%
[libx2

ℹ Pushing repo Hevagog/tqc-PandaPickAndPlace-v3 to the Hugging Face
Hub


...

actor.optimizer.pth:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

tqc-PandaPickAndPlace-v3.zip:   0%|          | 0.00/94.7M [00:00<?, ?B/s]

vec_normalize.pkl:   0%|          | 0.00/3.32k [00:00<?, ?B/s]

ℹ Your model is pushed to the Hub. You can view your model here:
https://huggingface.co/Hevagog/tqc-PandaPickAndPlace-v3/tree/main/


CommitInfo(commit_url='https://huggingface.co/Hevagog/tqc-PandaPickAndPlace-v3/commit/3839280f35cd0e38120e8ef12fbdf03599bf5d2a', commit_message='TQC upgrade for PandaPickAndPlace-v3', commit_description='', oid='3839280f35cd0e38120e8ef12fbdf03599bf5d2a', pr_url=None, pr_revision=None, pr_num=None)